# Concurrency

Definitions of concurrency:

* Concurrency is not the same as parallelism. 

* Concurrency is not a matter of application implementation. 

* Concurrency can be defined as follows: "Two events are concurrent if neither can causally affect the other."

* In other words, something is concurrent if it can be fully or partially decomposed into components (units) that are order-independent.

Common application scenarios where concurrent processing is a viable approach:

* Processing distribution: The scale of the problem is so big that the only way to process it in an acceptable time frame (with constrained resources) is to distribute execution on multiple processing units that can handle the work in parallel.

* Application responsiveness: Your application needs to maintain responsiveness (accept new inputs), even if it did not finish processing previous inputs.

* Background processing: Not every task needs to be performed in a synchronous way. If there is no need to immediately access the results of a specific action, it may be reasonable to defer execution in time.

Python's tools to deal with concurrency:

* Multithreading: 
    * Running multiple threads of execution that share the memory context of the parent process. 
    * The execution of threads is coordinated by the OS kernel.
    * It works best in applications that do a lot of I/O operations or need to maintain UI responsiveness. 
    * Very lightweight, but comes with caveats and memory safety risks.

* Multiprocessing: 
    * Running multiple independent processes to perform work in a distributed manner. 
    * Similar to threads in operation, but there's no shared memory context. 
    * Due to Python's GIL limitaton, it's better suited for CPU-intensive applications. 
    * More heavyweight than multithreading and requires implementing inter-process communication patterns to orchestrate work between processes.

* Asynchronous programming: 
    * Running multiple cooperative tasks within a single application process. 
    * Cooperative tasks work like threads, but switching between them is done by the application, not the OS kernel. 
    * Well suited to I/O-bound applications, especially for programs that need to handle multiple simultaneous network connections. 
    * The downside of asynchronous programming is the need to use dedicated asynchronous libraries.



### Multithreading

To start a new thread in Python, use the `threading.Thread()` class, as follows:


In [1]:
!python _01_multithreading/start_new_thread.py


Starting thread...
printing from thread
Ending thread...


From the last code example:

* To start a new thread, we call the `start()` method.

* Once the new thread is started, it'll run next to the main thread until the target function finishes. 

* To end the thread, we wait for the thread to finish with the `join()` method, which is a blocking operation.


With a small modification, we can also start and join multiple threads in bulk, as follows:


In [2]:
!python _01_multithreading/start_new_threads.py


Starting threads...

printing from thread
printing from threadprinting from thread

printing from thread
printing from thread
printing from threadprinting from thread

printing from thread
printing from thread
printing from thread
Ending threads...


Thread safety:

* All threads share the same memory context, so we must be careful when many threads access the same data structures. 

* If 2 parallel threads update the same variable without any protection, there might be a situation where a subtle timing variation in thread execution can alter the final result in an unexpected way. 

* Such situations are called `race conditions`, and they're very hard to debug. In those cases, the code is not 'thread-safe'.

For example, the following program is not thread-safe, and it returns a different value in every execution:


In [3]:
# The correct output is 100 threads * 100.000 iterations = 10.000.000

!python _01_multithreading/thread_visits.py 


thread_count=100, thread_visits=9299475


To avoid race conditions, we need to to use thread locking primitives. Python provides the `threading.Lock` class, which is a simple implementation of a thread lock. 

The following is an example of a thread-safe variant of the last code:



In [4]:
!python _01_multithreading/thread_safe_visits.py


thread_count=100, thread_visits=10000000


In the last code example, the thread visits with locks are counted properly, but at the expense of lower performance: 

* The lock will make sure that only 1 thread at a time can process a single block of code, so the protected block cannot run in parallel. 

* Also, acquiring and releasing locks are operations that add overhead.


#### Python's limitation with threads:

The standard implementation of Python (the CPython interpreter) comes with a major limitation that renders threads less useful in many contexts: 

* All operations accessing Python objects are serialized by the `Global Interpreter Lock (GIL)`.

* This is done because many of the interpreter's internal structures are not thread-safe and need to be protected. 

* Not every operation requires locking, and there are certain situations when threads release the lock:
    * In blocking system calls like socket calls.     
    * In sections of C extensions that do not use any Python/C API functions.

* So multiple threads can do I/O operations or execute some C extension code completely in parallel.
